This notebook contains evaluation of LongT5 Model trained on pubmed research paper dataset. ROUGE is used for measuring accuracy. The test example used is from BigPatent dataset.
https://huggingface.co/whaleloops/longt5-tglobal-large-16384-pubmed-10k_steps

This notebook has not been executed so scores are not mentioned.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a16ccdd52d987845800134b0360929df226c9c88a2815fbebfa0bdd3701d820b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff859

In [4]:
from datasets import load_dataset

#Mechanical Engineering; Lightning; Heating; Weapons; Blasting
ds = load_dataset("big_patent", "f")  # only 'f' CPC codes


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/85568 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4754 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4754 [00:00<?, ? examples/s]

Dataset big_patent downloaded and prepared to /root/.cache/huggingface/datasets/big_patent/f/2.1.2/bc8ec8bdf469c0da5fef04becd32bb3b0b34df0b0baa088ae1237628dd7a9caa. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
#import pandas as pd
# put data into dataFrames and csv
#df_train = pd.DataFrame(ds['train'])
#df_val = pd.DataFrame(ds['validation'])
#df_test = pd.DataFrame(ds['test'])
#df_train.to_csv("train.csv",index=False)
#df_val.to_csv("val.csv",index=False)
#df_test.to_csv("test.csv",index=False)

In [6]:
# cleaning text
import re

def remove_sentence_with_fig(paragraph):


    # first we remove . after FIG
    paragraph = re.sub(r"FIG.","FIG",paragraph)

    # Split the paragraph into sentences
    sentences = re.split(r'(?<=[.!?])\s+', paragraph)

    # Remove sentences containing "FIG."
    filtered_sentences = [sentence for sentence in sentences if "FIG" not in sentence]

    # Join the remaining sentences to form the updated paragraph
    updated_paragraph = ' '.join(filtered_sentences)

    return updated_paragraph

def clean_text(text):

    # replace ° degree with
    text = re.sub(r"°", " degree", text)

    # replace US Patent
    text = re.sub(r" U.S. Pat. No. [0-9,]+", "Patent", text)

    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)
    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)


    text = text.strip()

    return text


text = ds['test'][0]["description"]
cleaned_text = clean_text(text)
text = remove_sentence_with_fig(cleaned_text)

print(cleaned_text)
print(text)

BACKGROUND OF THE INVENTION This invention relates to a fuel economy running indicator which may be used during the driving operation of an automobile. In general, an internal combustion engine for a vehicle is designed to have a scope for operating at a constant gear ratio of speed change gears even if the load of the vehicle changes considerably. Accordingly, it is possible that a driver selects the gear ratio within the scope of a certain degree to operate a vehicle, such as a truck, having a number of speed change gears. However, under a constant power output, the internal combustion engine used for such a vehicle makes it possible to accomplish a satisfactory state of specific fuel consumption, that is, a fuel economy running state, by setting a suitable operation condition and, for this purpose, it is necessary to continually set a suitable gear ratio or proper travel of an accelerator pedal. This has become a substantial problem in this energy conserving age. Prior to the disclo

In [7]:
# divide the text into paragraphs of 16000
text1 = text.split()
text_lst = []
for i in range(0,len(text1),16000):
  tmp = " ".join(text1[i:i+16000])
  text_lst.append(tmp)


In [8]:
# exploring the dataset
print(ds['test'][0]["description"])

print("\n\n\n-------------")
print(ds['test'][0]["abstract"])

BACKGROUND OF THE INVENTION 
     This invention relates to a fuel economy running indicator which may be used during the driving operation of an automobile. 
     In general, an internal combustion engine for a vehicle is designed to have a scope for operating at a constant gear ratio of speed change gears even if the load of the vehicle changes considerably. Accordingly, it is possible that a driver selects the gear ratio within the scope of a certain degree to operate a vehicle, such as a truck, having a number of speed change gears. 
     However, under a constant power output, the internal combustion engine used for such a vehicle makes it possible to accomplish a satisfactory state of specific fuel consumption, that is, a fuel economy running state, by setting a suitable operation condition and, for this purpose, it is necessary to continually set a suitable gear ratio or proper travel of an accelerator pedal. This has become a substantial problem in this energy conserving age. 


In [9]:
# For cleaning predicted text
def clean_predicted_text(text):


    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)

    #replace new line character
    text = re.sub(r"<n>", "\n", text)

    #replace start of paragraph
    text = re.sub(r"<s>", "", text)

    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)




    return text

In [41]:
# creating model
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model/")

model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/model/")
# summary expected length
#expected_length = len(ds['test'][0]["abstract"])//2

summarizer = pipeline("summarization",model=model,tokenizer=tokenizer,num_beams=3,no_repeat_ngram_size=4,top_k=20)


In [12]:
# checkpoint = "EleutherAI/pythia-1.4b-deduped"
# assistant_checkpoint = "EleutherAI/pythia-160m-deduped"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# inputs = tokenizer(prompt, return_tensors="pt")

# model = AutoModelForCausalLM.from_pretrained(checkpoint)
# assistant_model = AutoModelForCausalLM.from_pretrained(assistant_checkpoint)
# outputs = model.generate(**inputs, assistant_model=assistant_model, do_sample=True, temperature=0.5)
# tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [42]:

#summarize data
#pred_summary_length = expected_length//len(text_lst)
predictions = []
for i in range(0,len(text_lst)):
  tmp = text_lst[i]

  prediction = clean_predicted_text(summarizer(tmp,min_length=200,max_length=1000,do_sample=True)[0]["summary_text"])
  #predictions
  predictions.append(prediction)


In [43]:
from rouge_score import rouge_scorer
abstract = ds['test'][0]["abstract"]
predicted_abstract = " ".join(predictions)
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
scores = scorer.score(abstract, predicted_abstract)

In [44]:
predictions

['Another object of the present invention is to provide a fuel economy running indicator indicating whether a combination of an accelerator pedal travelling amount determining the power output state of an internal combustion engine and a gear ratio of speed change gears, which are selected from some operating parameters varied by a driver, is within the scope of an economy fuel consumption. In the present invention, a fuel consumption running indicator is provided, which is based on a voltage at the neutral point of an AC generator, a voltage signal proportional to a travelling amount of the accelerator pedal with the Diesel engine of a truck, and a voltage value corresponding to an amount of travelling the accelerator pedal, which is inputted through the lead 3a, by a comparator 4 which outputs a voltage on a lead 4a when the voltage value on the lead 2b becomes larger than that on the lead 4a. According to this invention, the voltage value at the output of the comparator 4 is compare

In [40]:
#rouge max_length tokens 100
scores


{'rouge1': Score(precision=0.3854166666666667, recall=0.48366013071895425, fmeasure=0.42898550724637685),
 'rouge2': Score(precision=0.12041884816753927, recall=0.1513157894736842, fmeasure=0.13411078717201166),
 'rougeL': Score(precision=0.19270833333333334, recall=0.24183006535947713, fmeasure=0.21449275362318843)}